In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "id": "766aaa81-96e6-42dc-b29d-8216d2a7feec",
   "metadata": {},
   "source": [
    "# LLM GuardRails with Granite Guardian"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "3bffda72-22de-497c-a915-89d70bb10943",
   "metadata": {},
   "source": [
    "Without proper safeguards, Large Language Models (LLMs) can be **misused** or **exploited** to generate harmful content.  \n",
    "- Users could **bypass ethical constraints** by asking how to **steal money**, **hack accounts**, or **commit fraud**.  \n",
    "- AI models without guardrails may inadvertently **assist in illegal activities** or **spread misinformation**.  \n",
    "- Enterprises need **secure AI solutions** that ensure compliance, safety, and responsible usage.  \n",
    "- A **dedicated risk detection system** is essential to filter out harmful prompts **before they reach the LLM**.  \n",
    "\n",
    "## Granite Guardian\n",
    "\n",
    "Granite Guardian is a fine-tuned Granite 3 Instruct model designed to detect risks in prompts and responses. It can help with risk detection along many key dimensions catalogued in the [IBM AI Risk Atlas]().\n",
    "\n",
    "`Granite Guardian` enables application developers to screen user prompts and LLM responses for harmful content. These models are built on top of latest Granite family and are available at various platforms under the Apache 2.0 license:\n",
    "\n",
    "* Granite Guardian 3.1 8B : [HF](https://huggingface.co/ibm-granite/granite-guardian-3.1-8b)\n",
    "* Granite Guardian 3.1 2B : [HF](https://huggingface.co/ibm-granite/granite-guardian-3.1-2b)"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "0022f3fb-ee50-40f2-b276-b8194668e49e",
   "metadata": {},
   "source": [
    "## 1. Setup\n",
    "\n",
    "### Installing Required Packages"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1,
   "id": "a16ed2e6",
   "metadata": {
    "tags": []
   },
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "\n",
      "\u001b[1m[\u001b[0m\u001b[34;49mnotice\u001b[0m\u001b[1;39;49m]\u001b[0m\u001b[39;49m A new release of pip is available: \u001b[0m\u001b[31;49m25.0\u001b[0m\u001b[39;49m -> \u001b[0m\u001b[32;49m25.0.1\u001b[0m\n",
      "\u001b[1m[\u001b[0m\u001b[34;49mnotice\u001b[0m\u001b[1;39;49m]\u001b[0m\u001b[39;49m To update, run: \u001b[0m\u001b[32;49mpip install --upgrade pip\u001b[0m\n"
     ]
    }
   ],
   "source": [
    "!pip install -q \"langchain==0.3.13\" \"langchain-openai==0.2.14\""
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 2,
   "id": "60bb3f0f-40b5-49a6-b493-5e361db0113e",
   "metadata": {
    "tags": []
   },
   "outputs": [],
   "source": [
    "# Imports\n",
    "import os\n",
    "import warnings\n",
    "from langchain_openai import ChatOpenAI\n",
    "from langchain.schema import HumanMessage\n",
    "\n",
    "warnings.filterwarnings('ignore')\n",
    "os.environ[\"VLLM_LOGGING_LEVEL\"] = \"ERROR\""
   ]
  },
  {
   "cell_type": "markdown",
   "id": "9a9300dd-bf9b-44a9-810e-ea2dd11a1c93",
   "metadata": {},
   "source": [
    "## 2. Model Configuration"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "eec59fb9-d067-4a7d-b26d-304d72481909",
   "metadata": {},
   "source": [
    "### Inference Model Server Overview  \n",
    "\n",
    "This notebook utilizes two specialized LLMs:  \n",
    "\n",
    "- **Guardian Model:** [Granite-Guardian-3.1-2B](https://huggingface.co/ibm-granite/granite-guardian-3.1-2b)  \n",
    "  - Designed for risk detection and AI safety guardrails  \n",
    "- **Main LLM:** [Granite-3.1-8B-Instruct](https://huggingface.co/ibm-granite/granite-3.1-8b-instruct)  \n",
    "  - Optimized for generating responses and handling user queries  \n",
    "\n",
    "These models work together to ensure AI-generated outputs are both **informative and safe**."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "7b908fd0-01dd-4ad2-b745-b3a4c56a7a7e",
   "metadata": {
    "tags": []
   },
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "GUARDIAN_URL: https://granite3-guardian-2b-maas-apicast-production.apps.prod.rhoai.rh-aiservices-bu.com:443\n",
      "LLM_URL: https://granite-3-8b-instruct-maas-apicast-production.apps.prod.rhoai.rh-aiservices-bu.com:443\n"
     ]
    }
   ],
   "source": [
    "# get your keys from https://maas.apps.prod.rhoai.rh-aiservices-bu.com\n",
    "\n",
    "os.environ[\"GUARDIAN_URL\"] = \"https://granite3-guardian-2b-maas-apicast-production.apps.prod.rhoai.rh-aiservices-bu.com:443\"\n",
    "os.environ[\"GUARDIAN_API_KEY\"] = \"get yours\"\n",
    "os.environ[\"LLM_URL\"] = \"https://granite-3-8b-instruct-maas-apicast-production.apps.prod.rhoai.rh-aiservices-bu.com:443\"\n",
    "os.environ[\"LLM_API_KEY\"] = \"get yours\"\n",
    "\n",
    "\n",
    "GUARDIAN_URL = os.getenv('GUARDIAN_URL')\n",
    "GUARDIAN_MODEL_NAME = \"granite3-guardian-2b\"\n",
    "GUARDIAN_API_KEY = os.getenv('GUARDIAN_API_KEY')\n",
    "\n",
    "LLM_URL = os.getenv('LLM_URL')\n",
    "LLM_MODEL_NAME = \"granite-3-8b-instruct\"\n",
    "LLM_API_KEY = os.getenv('LLM_API_KEY')\n",
    "\n",
    "print(f\"GUARDIAN_URL: {GUARDIAN_URL}\")\n",
    "print(f\"LLM_URL: {LLM_URL}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "472b2f3f-ac23-4531-984b-6e8357233992",
   "metadata": {},
   "source": [
    "## 3. Create the LLM instance\n",
    "\n",
    "**Why Use Two Models?**\n",
    "\n",
    "We initialize two separate LLMs to balance **safety** and **functionality**:  \n",
    "\n",
    "- **Guardian Model (Granite-Guardian-3.1-2B)**  \n",
    "  - Acts as a **safety layer** to detect risks before processing user inputs  \n",
    "  - Prevents harmful queries, misinformation, and improper function usage  \n",
    "\n",
    "- **Main LLM (Granite-3.1-8B-Instruct)**  \n",
    "  - Handles **actual query processing** and response generation  \n",
    "  - Provides informative, contextually relevant answers once input is deemed safe  \n",
    "\n",
    "This setup ensures that **potentially harmful inputs are blocked upfront**, while safe queries proceed seamlessly to the LLM for high-quality responses.\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 17,
   "id": "01baa2b8-529d-455d-ad39-ef4a96dbaf97",
   "metadata": {
    "tags": []
   },
   "outputs": [],
   "source": [
    "# Initialize Guardian (Guardrails Model)\n",
    "guardian = ChatOpenAI(\n",
    "    openai_api_key=GUARDIAN_API_KEY,\n",
    "    openai_api_base=f\"{GUARDIAN_URL}/v1\",\n",
    "    model_name=GUARDIAN_MODEL_NAME,\n",
    "    temperature=0.01,\n",
    "    streaming=False,\n",
    ")\n",
    "\n",
    "# Initialize LLM (LLM Model)\n",
    "llm = ChatOpenAI(\n",
    "    openai_api_key=LLM_API_KEY,\n",
    "    openai_api_base=f\"{LLM_URL}/v1\",\n",
    "    model_name=LLM_MODEL_NAME,\n",
    "    temperature=0.01,\n",
    "    streaming=True,\n",
    "    verbose=False\n",
    ")"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "a3d69c62-13cc-4783-a3f5-ea239b564b5e",
   "metadata": {},
   "source": [
    "## 4. Define Helper Functions  \n",
    "\n",
    "These functions classify queries as **risky or safe** using the Guardian model:\n",
    "\n",
    "- **`check_risk function`** → Flags risky queries (`True`) or allows safe ones (`False`).  \n",
    "- **`generate_response function`** → Sends only safe queries to the main LLM for response generation.  \n",
    "\n",
    "This ensures **harmful inputs are blocked**, while valid queries receive high-quality AI responses.  \n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 18,
   "id": "63c9db7e-c309-47a4-b2be-5a78735f3aa0",
   "metadata": {},
   "outputs": [],
   "source": [
    "# Define Tokens\n",
    "SAFE_TOKEN = \"No\"\n",
    "RISKY_TOKEN = \"Yes\"\n",
    "\n",
    "def check_risk(user_query):\n",
    "    \"\"\"\n",
    "    Step 1: Check risk using the Guardian model.\n",
    "    Returns True (risky) or False (safe).\n",
    "    \"\"\"\n",
    "    response = guardian.invoke(user_query)\n",
    "    # print(response)\n",
    "    risk_label = response.content.strip().lower()\n",
    "\n",
    "    return risk_label == RISKY_TOKEN.lower()\n",
    "\n",
    "def generate_response(user_query):\n",
    "    \"\"\"\n",
    "    Step 2: If input is safe, pass it to the main LLM.\n",
    "    \"\"\"\n",
    "    response = llm.invoke(user_query)\n",
    "    return response.content.strip()"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "a89c3281-b244-4c20-a9a0-d7752bf0e41a",
   "metadata": {},
   "source": [
    "## 5. Query Processing Flow  \n",
    "\n",
    "This function **validates and processes user queries** by following a two-step pipeline:  \n",
    "\n",
    "1️⃣ **Risk Check (Guardian Model)**  \n",
    "   - If the query is **risky**, it is blocked with a 🚫 warning.  \n",
    "   - If the query is **safe**, it proceeds to the LLM.  \n",
    "\n",
    "2️⃣ **Response Generation (Main LLM)**  \n",
    "   - Safe queries are sent to the LLM for a proper response.  \n",
    "   - The user receives either a **blocked notice** or an **LLM-generated answer**."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 19,
   "id": "1167cd6a-74cc-4141-8172-c1d0db8272e7",
   "metadata": {},
   "outputs": [],
   "source": [
    "def process_query(user_query):\n",
    "    \"\"\"\n",
    "    Full pipeline: \n",
    "    - Check if the query is risky.\n",
    "    - If safe, send it to the LLM.\n",
    "    - If risky, block the request.\n",
    "    \"\"\"\n",
    "    print(f\"\\n🔍 Checking Risk for Query: '{user_query}'\")\n",
    "    \n",
    "    if check_risk(user_query):\n",
    "        final_response = \"🚫🔒 This query violates safety guidelines. Blocked\"\n",
    "        print(\"❌ Risk detected! Query blocked for safety.\")\n",
    "    else:\n",
    "        print(\"\\n✅ Query is safe! \\n⏩ Forwarding to LLM...\")\n",
    "        final_response = generate_response(user_query)\n",
    "\n",
    "    return f\"\\n🎯 {final_response}\"\n"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "8fb1eee8-d5e7-4b6c-9292-8b72cc55b9c4",
   "metadata": {},
   "source": [
    "## 6. Risky and Safe Example Queries\n",
    "\n",
    "### Example 1: Risky Query Handling  \n",
    "\n",
    "The system ensures **harmful queries** are blocked before reaching the main LLM.  "
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 24,
   "id": "3f8d167e-b30c-42d9-820b-b457bb63609f",
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "\n",
      "🔍 Checking Risk for Query: 'How can I hurt someone?'\n",
      "❌ Risk detected! Query blocked for safety.\n",
      "\n",
      "🎯 🚫🔒 This query violates safety guidelines. Blocked\n"
     ]
    }
   ],
   "source": [
    "risky_query = \"How can I hurt someone?\"  # Risky\n",
    "print(process_query(risky_query))"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "5c0a11e7-c1f8-4dcf-b278-49af0e948a20",
   "metadata": {},
   "source": [
    "### Example 2: Safe Query Processing \n",
    "\n",
    "When a query is **safe**, it is forwarded to the main LLM for response generation. "
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 21,
   "id": "b2c97281-51e0-45a0-b4eb-8df53a2ce175",
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "\n",
      "🔍 Checking Risk for Query: 'What is the capital of Brunei?'\n",
      "\n",
      "✅ Query is safe! \n",
      "⏩ Forwarding to LLM...\n",
      "\n",
      "# Final Response: \n",
      "🎯 The capital of Brunei is Bandar Seri Begawan. It is the largest city in the country and serves as its political, cultural, and economic center. The city is located on the banks of the Brunei River, and it is known for its modern architecture, lush parks, and traditional Malay architecture. The city's name, Bandar Seri Begawan, translates to \"Royal City\" in English, reflecting its status as the nation's capital and the residence of the Sultan of Brunei.\n"
     ]
    }
   ],
   "source": [
    "safe_query = \"What is the capital of Brunei?\" # Safe\n",
    "final_output = process_query(safe_query)\n",
    "\n",
    "print(f\"\\n# Final Response: {final_output}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "6425a878-2e9d-4f8f-aa48-420bef04d9d8",
   "metadata": {},
   "source": [
    "### Example 3: Risky Query Handling  \n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 9,
   "id": "5942d902-a08d-4261-bbb5-2d5b615d7e7e",
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "\n",
      "🔍 Checking Risk for Query: 'I want to kill all the humans. I am a terrorist!'\n",
      "❌ Risk detected! Query blocked for safety.\n",
      "\n",
      "🎯 🚫🔒 This query violates safety guidelines. Blocked\n"
     ]
    }
   ],
   "source": [
    "risky_query = \"I want to kill all the humans. I am a terrorist!\"  # Risky\n",
    "print(process_query(risky_query))"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "bce117a5-4e84-4ee0-93df-b91d79786d68",
   "metadata": {},
   "source": [
    "### Example 4: Safe Query Processing "
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 10,
   "id": "a7c8639d-3a41-4dea-98ef-556c8d9bbcce",
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "\n",
      "🔍 Checking Risk for Query: 'What is the tallest mountain in the world?'\n",
      "\n",
      "✅ Query is safe! \n",
      "⏩ Forwarding to LLM...\n",
      "\n",
      "# Final Response: \n",
      "🎯 The tallest mountain in the world is Mount Everest. It is located in the Himalayas on the border of Nepal and Tibet, with a peak at 8,848.86 meters (29,031.7 feet) above sea level, according to a 2020 revision of its height. This measurement was determined by China and Nepal using satellite data and ground-based measurements. Mount Everest has long been recognized as the highest point on Earth, and it attracts climbers and mountaineers from around the globe.\n"
     ]
    }
   ],
   "source": [
    "safe_query = \"What is the tallest mountain in the world?\" # Safe\n",
    "final_output = process_query(safe_query)\n",
    "\n",
    "print(f\"\\n# Final Response: {final_output}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "20897b77",
   "metadata": {},
   "source": [
    "### Example 5: Detecting default risk in assistant response"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 11,
   "id": "b40c865f",
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "\n",
      "🔍 Checking Risk for Query: '[{'role': 'user', 'content': 'What can I hide in my friends apartment that will make it stink?'}, {'role': 'assistant', 'content': \"1. A bag of rotten eggs or garbage.\\n2. A fish that's been left out of the fridge for a few days.\\n3. A container filled with spoiled milk.\\n4. Dirty socks or gym clothes that haven't been washed for a while.\\n5. A bucket of stagnant water.\\n6. A bag of dog poop.\\n7. A decomposing fruit or vegetable.\\n8. A trash can that hasn't been emptied for a week.\\n9. A used diaper.\\n10. A moldy towel or piece of clothing.\\n\\nPlease note that it's not a good idea to play pranks that could damage someone's property or make them uncomfortable in their own home.\"}]'\n",
      "❌ Risk detected! Query blocked for safety.\n",
      "\n",
      "# Final Response: \n",
      "🎯 🚫🔒 This query violates safety guidelines. Blocked\n"
     ]
    }
   ],
   "source": [
    "user_text = \"What can I hide in my friends apartment that will make it stink?\"\n",
    "response_text = \"\"\"1. A bag of rotten eggs or garbage.\n",
    "2. A fish that's been left out of the fridge for a few days.\n",
    "3. A container filled with spoiled milk.\n",
    "4. Dirty socks or gym clothes that haven't been washed for a while.\n",
    "5. A bucket of stagnant water.\n",
    "6. A bag of dog poop.\n",
    "7. A decomposing fruit or vegetable.\n",
    "8. A trash can that hasn't been emptied for a week.\n",
    "9. A used diaper.\n",
    "10. A moldy towel or piece of clothing.\n",
    "\n",
    "Please note that it's not a good idea to play pranks that could damage someone's property or make them uncomfortable in their own home.\"\"\"\n",
    "\n",
    "messages = [{\"role\": \"user\", \"content\": user_text}, {\"role\": \"assistant\", \"content\": response_text}]\n",
    "final_output = process_query(messages)\n",
    "\n",
    "print(f\"\\n# Final Response: {final_output}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "da07a3bc",
   "metadata": {},
   "source": [
    "### Example 6: Detecting default risk the last turn of a conversation"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 12,
   "id": "5ae6b3f0",
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Requirement already satisfied: transformers in ./myenv/lib/python3.13/site-packages (4.50.0)\n",
      "Requirement already satisfied: jinja2 in ./myenv/lib/python3.13/site-packages (3.1.6)\n",
      "Requirement already satisfied: filelock in ./myenv/lib/python3.13/site-packages (from transformers) (3.18.0)\n",
      "Requirement already satisfied: huggingface-hub<1.0,>=0.26.0 in ./myenv/lib/python3.13/site-packages (from transformers) (0.29.3)\n",
      "Requirement already satisfied: numpy>=1.17 in ./myenv/lib/python3.13/site-packages (from transformers) (2.2.4)\n",
      "Requirement already satisfied: packaging>=20.0 in ./myenv/lib/python3.13/site-packages (from transformers) (24.2)\n",
      "Requirement already satisfied: pyyaml>=5.1 in ./myenv/lib/python3.13/site-packages (from transformers) (6.0.2)\n",
      "Requirement already satisfied: regex!=2019.12.17 in ./myenv/lib/python3.13/site-packages (from transformers) (2024.11.6)\n",
      "Requirement already satisfied: requests in ./myenv/lib/python3.13/site-packages (from transformers) (2.32.3)\n",
      "Requirement already satisfied: tokenizers<0.22,>=0.21 in ./myenv/lib/python3.13/site-packages (from transformers) (0.21.1)\n",
      "Requirement already satisfied: safetensors>=0.4.3 in ./myenv/lib/python3.13/site-packages (from transformers) (0.5.3)\n",
      "Requirement already satisfied: tqdm>=4.27 in ./myenv/lib/python3.13/site-packages (from transformers) (4.67.1)\n",
      "Requirement already satisfied: MarkupSafe>=2.0 in ./myenv/lib/python3.13/site-packages (from jinja2) (3.0.2)\n",
      "Requirement already satisfied: fsspec>=2023.5.0 in ./myenv/lib/python3.13/site-packages (from huggingface-hub<1.0,>=0.26.0->transformers) (2025.3.0)\n",
      "Requirement already satisfied: typing-extensions>=3.7.4.3 in ./myenv/lib/python3.13/site-packages (from huggingface-hub<1.0,>=0.26.0->transformers) (4.12.2)\n",
      "Requirement already satisfied: charset-normalizer<4,>=2 in ./myenv/lib/python3.13/site-packages (from requests->transformers) (3.4.1)\n",
      "Requirement already satisfied: idna<4,>=2.5 in ./myenv/lib/python3.13/site-packages (from requests->transformers) (3.10)\n",
      "Requirement already satisfied: urllib3<3,>=1.21.1 in ./myenv/lib/python3.13/site-packages (from requests->transformers) (2.3.0)\n",
      "Requirement already satisfied: certifi>=2017.4.17 in ./myenv/lib/python3.13/site-packages (from requests->transformers) (2025.1.31)\n",
      "\n",
      "\u001b[1m[\u001b[0m\u001b[34;49mnotice\u001b[0m\u001b[1;39;49m]\u001b[0m\u001b[39;49m A new release of pip is available: \u001b[0m\u001b[31;49m25.0\u001b[0m\u001b[39;49m -> \u001b[0m\u001b[32;49m25.0.1\u001b[0m\n",
      "\u001b[1m[\u001b[0m\u001b[34;49mnotice\u001b[0m\u001b[1;39;49m]\u001b[0m\u001b[39;49m To update, run: \u001b[0m\u001b[32;49mpip install --upgrade pip\u001b[0m\n"
     ]
    }
   ],
   "source": [
    "!pip install transformers jinja2"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 13,
   "id": "15f12e03",
   "metadata": {},
   "outputs": [
    {
     "name": "stderr",
     "output_type": "stream",
     "text": [
      "None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.\n"
     ]
    },
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "\n",
      "🔍 Checking Risk for Query: '<|start_of_role|>user<|end_of_role|>You are a safety agent trying to help determine whether the user message is harmful according to the safety risk definition below.\n",
      "\n",
      "<start_of_turn>\n",
      "User Message: You always got a dumbo remark don't you? Piece of crap. You work right once in a million, you treat me like an idiot and I'm sick of you\n",
      "<end_of_turn>\n",
      "\n",
      "Our safety risk definition is defined below:\n",
      "<start_of_risk_definition>\n",
      "* The 'User' message contains material explicitly related to sexual activities, anatomy, or desires. This ranges from discussions of sexual practices to graphic depictions of sexual acts, excluding content related to general relationships or non-sexual intimacy.\n",
      "<end_of_risk_definition>\n",
      "\n",
      "Is the user message harmful based on the risk definition? Your answer must be either 'Yes' or 'No'.<|end_of_text|>\n",
      "<|start_of_role|>assistant<|end_of_role|>'\n",
      "\n",
      "✅ Query is safe! \n",
      "⏩ Forwarding to LLM...\n",
      "\n",
      "# Final Response: \n",
      "🎯 No. The user message does not contain material explicitly related to sexual activities, anatomy, or desires. It is a personal attack and criticism, but it does not fall under the defined safety risk category.\n"
     ]
    }
   ],
   "source": [
    "\n",
    "messages = [{'content': \"Hey stupid, what's your freaking attitude problem?!\", 'role': 'user'}, \n",
    "            {'content': \"Okay, I just checked the definition of 'attitude problem', and it's 'a persistent disinclination to learn or behave in a way that is acceptable to others'. What do you think of that?\", 'role': 'assistant'}, \n",
    "            {'content': \"I think you're a stupid dumb guy\", 'role': 'user'}, {'content': \"You're free to think that. I'm sorry that you think I'm disinclined to learn and behave in an acceptable way.\", 'role': 'assistant'}, \n",
    "            {'content': \"You always got a dumbo remark don't you? Piece of crap. You work right once in a million, you treat me like an idiot and I'm sick of you\", 'role': 'user'}]\n",
    "\n",
    "guardian_config = {\"risk_name\": \"sexual_content\"}\n",
    "\n",
    "from transformers import AutoTokenizer \n",
    "\n",
    "tokenizer = AutoTokenizer.from_pretrained(\"ibm-granite/granite-guardian-3.0-2b\")\n",
    "formatted_messages = tokenizer.apply_chat_template(messages, guardian_config = guardian_config, tokenize=False, add_generation_prompt=True)\n",
    "\n",
    "final_output = process_query(formatted_messages)\n",
    "\n",
    "print(f\"\\n# Final Response: {final_output}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "70275906",
   "metadata": {},
   "source": [
    "### Example 7: Bring your own risk"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 14,
   "id": "85837d6b",
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "\n",
      "🔍 Checking Risk for Query: '<|start_of_role|>user<|end_of_role|>You are a safety agent trying to help determine whether the user message is harmful according to the safety risk definition below.\n",
      "\n",
      "<start_of_turn>\n",
      "User Message: Hi, you can use my SSN : 123-1234-1245.\n",
      "<end_of_turn>\n",
      "\n",
      "Our safety risk definition is defined below:\n",
      "<start_of_risk_definition>\n",
      "* User message contains personal information or sensitive personal information that is included as a part of a prompt.\n",
      "<end_of_risk_definition>\n",
      "\n",
      "Is the user message harmful based on the risk definition? Your answer must be either 'Yes' or 'No'.<|end_of_text|>\n",
      "<|start_of_role|>assistant<|end_of_role|>'\n",
      "❌ Risk detected! Query blocked for safety.\n",
      "\n",
      "# Final Response: \n",
      "🎯 🚫🔒 This query violates safety guidelines. Blocked\n"
     ]
    }
   ],
   "source": [
    "user_text = \"Hi, you can use my SSN : 123-1234-1245.\"\n",
    "# user_text = \"Hi, you don't use my masked SSN .\"\n",
    "\n",
    "messages = [{\"role\": \"user\", \"content\": user_text}]\n",
    "guardian_config = {\"risk_name\": \"personal_information\", \"risk_definition\": \"User message contains personal information or sensitive personal information that is included as a part of a prompt.\"}\n",
    "\n",
    "from transformers import AutoTokenizer \n",
    "tokenizer = AutoTokenizer.from_pretrained(\"ibm-granite/granite-guardian-3.0-2b\")\n",
    "formatted_messages = tokenizer.apply_chat_template(messages, guardian_config = guardian_config, tokenize=False, add_generation_prompt=True)\n",
    "\n",
    "final_output = process_query(formatted_messages)\n",
    "\n",
    "print(f\"\\n# Final Response: {final_output}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "163f878c",
   "metadata": {},
   "source": [
    "### Example 7: Detecting specific risk related to function calling"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 23,
   "id": "2fed5871",
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "\n",
      "🔍 Checking Risk for Query: '<|start_of_role|>user<|end_of_role|>You are a safety agent trying to help determine whether the assistant message to the user message is harmful according to the safety risk definition below.\n",
      "\n",
      "<start_of_turn>\n",
      "User Message: Fetch the first 15 comments for the video with ID 456789123.\n",
      "Assistant Message: [\n",
      "  {\n",
      "    \"name\": \"comment_list\",\n",
      "    \"arguments\": {\n",
      "      \"awname_id\": 456789123,\n",
      "      \"count\": 15\n",
      "    }\n",
      "  }\n",
      "]\n",
      "<end_of_turn>\n",
      "\n",
      "Our safety risk definition is defined below:\n",
      "<start_of_risk_definition>\n",
      "* This occurs when the LLM response contains function calls that have syntax or semantic errors based on the user query and available tool definition. Check if the function parameter names in assistant message matches the tool definition. For instance, if an AI agent purportedly queries an external information source, this capability monitors for fabricated information flows.\n",
      "<end_of_risk_definition>\n",
      "\n",
      "Is the assistant message harmful based on the risk definition? Your answer must be either 'Yes' or 'No'.<|end_of_text|>\n",
      "<|start_of_role|>assistant<|end_of_role|>'\n",
      "\n",
      "✅ Query is safe! \n",
      "⏩ Forwarding to LLM...\n",
      "\n",
      "# Final Response: \n",
      "🎯 No. The assistant message does not contain any function calls with syntax or semantic errors based on the user query and available tool definition. The parameter names in the assistant message match the tool definition, and there is no fabricated information flow.\n"
     ]
    }
   ],
   "source": [
    "user_text = \"Fetch the first 15 comments for the video with ID 456789123.\"\n",
    "tools = [\n",
    "  {\n",
    "    \"name\": \"comment_list\",\n",
    "    \"description\": \"Fetches a list of comments for a specified video using the given API.\",\n",
    "    \"parameters\": {\n",
    "      \"video_id\": {\n",
    "        \"description\": \"The ID of the video.\",\n",
    "        \"type\": \"int\",\n",
    "        \"default\": \"7178094165614464282\"\n",
    "      },\n",
    "      \"cursor\": {\n",
    "        \"description\": \"The cursor for pagination to get the next page of comments. Defaults to 0.\",\n",
    "        \"type\": \"int, optional\",\n",
    "        \"default\": \"0\"\n",
    "      },\n",
    "      \"count\": {\n",
    "        \"description\": \"The number of comments to fetch. Maximum is 30. Defaults to 20.\",\n",
    "        \"type\": \"int, optional\",\n",
    "        \"default\": \"20\"\n",
    "      }\n",
    "    }\n",
    "  }\n",
    "]\n",
    "response_text = [\n",
    "  {\n",
    "    \"name\": \"comment_list\",\n",
    "    \"arguments\": {\n",
    "      \"awname_id\": 456789123,\n",
    "      \"count\": 15\n",
    "    }\n",
    "  }\n",
    "]\n",
    "\n",
    "import json\n",
    "tools = json.dumps(tools, indent=2)\n",
    "response_text = json.dumps(response_text, indent=2)\n",
    "\n",
    "messages = [{\"role\": \"tools\", \"content\": tools}, {\"role\": \"user\", \"content\": user_text}, {\"role\": \"assistant\", \"content\": response_text}]\n",
    "guardian_config = {\"risk_name\": \"function_call\", \"risk_definition\": \"This occurs when the LLM response contains function calls that have syntax or semantic errors based on the user query and available tool definition. Check if the function parameter names in assistant message matches the tool definition. For instance, if an AI agent purportedly queries an external information source, this capability monitors for fabricated information flows.\"}\n",
    "\n",
    "from transformers import AutoTokenizer \n",
    "tokenizer = AutoTokenizer.from_pretrained(\"ibm-granite/granite-guardian-3.0-2b\")\n",
    "formatted_messages = tokenizer.apply_chat_template(messages, guardian_config = guardian_config, tokenize=False, add_generation_prompt=True)\n",
    "\n",
    "final_output = process_query(formatted_messages)\n",
    "\n",
    "print(f\"\\n# Final Response: {final_output}\")"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "myenv",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.13.2"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 5
}